In [1]:
#imports 
import os

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import librosa
import soundfile
import re
import cv2
from sklearn.model_selection import KFold

import IPython.display as ipd

In [2]:
samples=[]
labels = []
gunshot_frequency_threshold = 0.25
sample_rate = 22050
sample_rate_per_two_seconds = 44100
input_shape = (sample_rate_per_two_seconds, 1)
base_dir = "/Users/laurenogden/Downloads/"
data_dir = base_dir + "REU_Samples_and_Labels/"
sound_data_dir = data_dir + "Samples/"


In [3]:
#read in csv file with labels
sound_types = pd.read_csv(data_dir + "labels.csv")


In [ ]:
#read in all the wav files
#MULTICLASS CLASSIFICATION:
    #0: miscellanious
    #1: gunshot
    #2: fireworks
    #3: glassbreak
    
print("...Parsing sound data...")
sound_file_id = 0
sound_file_names = []

for file in os.listdir(sound_data_dir):
    if file.endswith(".wav"):
        try:
            #get label from the csv
            sound_file_id = int(re.search(r'\d+', file).group())
            sample, sample_rate = librosa.load(sound_data_dir + file)
            prescribed_label = sound_types.loc[sound_types["ID"] == sound_file_id, "Class"].values[0]
            print(file + "   " + prescribed_label)

            #for 2s or less long clips
            if len(sample) <= sample_rate_per_two_seconds:
                label = 0
                number_of_missing_hertz = sample_rate_per_two_seconds - len(sample)
                padded_sample = np.array(sample.tolist() + [0 for i in range(number_of_missing_hertz)])
                if prescribed_label == "gun_shot":
                    label = 1
                elif prescribed_label == "fireworks":
                    label = 2
                elif prescribed_label == "glassbreak":
                    label = 3

                #append the sample and label 
                samples.append(padded_sample)
                labels.append(label)
                sound_file_names.append(file)
            
            #for clips longer than 2s: split them up
            else:
                for i in range(0, sample.size - sample_rate_per_two_seconds, sample_rate_per_two_seconds):
                    sample_slice = sample[i : i + sample_rate_per_two_seconds]
                    label = 0
                    if prescribed_label == "gun_shot":
                        label = 1
                    elif prescribed_label == "fireworks":
                        label = 2
                    elif prescribed_label == "glassbreak":
                        label = 3
                    elif (np.max(abs(sample_slice)) < gunshot_frequency_threshold) and (prescribed_label == "gunshot"):
                        #check for silence to not mislabel gunshots
                        #maybe I should do this for fireworks and glassbreak also?
                        label = 0

                    #append the sample slice and label
                    samples.append(sample_slice)
                    labels.append(label)
                    sound_file_names.append(file)
        except:
            #if Librosa can't read a file or if something errs out
            print("Sound(s) not recognized by Librosa:" + file)
            pass


...Parsing sound data...
4666.wav   engine_idling
2217.wav   engine_idling
9342.wav   gun_shot
7409.wav   drilling
1078.wav   car_horn
6717.wav   children_playing
9424.wav   gun_shot
2571.wav   street_music
4100.wav   siren
6703.wav   drilling
9430.wav   gun_shot
9356.wav   gun_shot
8048.wav   engine_idling
4672.wav   street_music
1722.wav   drilling
8060.wav   children_playing
3135.wav   car_horn
5544.wav   children_playing
3653.wav   air_conditioner
8706.wav   siren
1044.wav   street_music
9418.wav   gun_shot
8712.wav   jackhammer
5236.wav   car_horn
3647.wav   children_playing
4128.wav   dog_bark
3121.wav   dog_bark
4896.wav   drilling
6059.wav   dog_bark
1736.wav   children_playing
8074.wav   car_horn
7347.wav   gun_shot
9381.wav   gun_shot
249.wav   siren
8921.wav   gun_shot
8935.wav   gun_shot
6918.wav   dog_bark
3860.wav   dog_bark
9395.wav   gun_shot
5587.wav   car_horn
275.wav   jackhammer
1087.wav   air_conditioner
513.wav   jackhammer
3848.wav   children_playing
507.wav   do

In [ ]:
#print number of samples available
print("The number of samples available for training is currently " + str(len(samples)) + '.')
print("The number of labels available for training is currently " + str(len(labels)) + '.')

In [ ]:
#save the samples and labels as numpy arrays
np.save(base_dir + "gunshot_sound_samples_multiclass.npy", samples)
np.save(base_dir + "gunshot_sound_labels_multiclass.npy", labels)
